In [53]:
from Strategies.StrategyBB import StrategyBB
from Strategies.StrategyMACD import StrategyMACD
from Strategies.StrategyOBV import StrategyOBV
from Strategies.StrategyRSI import StrategyRSI
from Strategies.StrategyST import StrategyST
from historyData.HistoryData import HistoryData
from tinkoff.invest.grpc.marketdata_pb2 import CandleInterval
from datetime import datetime

import asyncio

In [54]:
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN
data_set = HistoryData()

In [66]:
from time import sleep

candles_down = None
candles_up = None
candles_flat = None
candles_short_up = None
candles_short_down = None

async def data_get():
    global candles_down, candles_up, candles_flat, candles_short_up, candles_short_down
    
    candles_short_up = await (data_set.get_tinkoff_server_data(datetime(2023, 1, 1), datetime(2023, 2, 1), candle_interval))
    candles_short_down = await (data_set.get_tinkoff_server_data(datetime(2021, 1, 1), datetime(2021, 2, 1), candle_interval))
    candles_down = await (data_set.get_tinkoff_server_data(datetime(2022, 2, 1), datetime(2022, 3, 1), candle_interval))
    candles_up = await (data_set.get_tinkoff_server_data(datetime(2023, 3, 1), datetime(2023, 4, 1), candle_interval))
    candles_flat = await (data_set.get_tinkoff_server_data(datetime(2024, 1, 1), datetime(2024, 2, 1), candle_interval))

# Run the event loop directly using await
await data_get()

# Creates a Data Set For Future Model

In [71]:
all_candles = [candles_down, candles_up, candles_flat, candles_short_up, candles_short_down]
len(all_candles[3])

21830

In [72]:
from AppLogic.Settings import Settings
from tinkoff.invest.utils import quotation_to_decimal

commission = 0.0005
all_data_index_buy_sell = []

for candles in all_candles:
    data_index_buy_sell = []
    total_earn = 0
    total_sum_earned = 0
    bought_at = -1
    for i, candle in enumerate(list(candles)):
        bought_at = float(quotation_to_decimal(candle.close))
        this_candle_earned = 0
        for j in range(i + 1, len(candles)):
            further_candle_price = float(quotation_to_decimal(candles[j].close))
            this_candle_earned = bought_at - further_candle_price
            if this_candle_earned > 0.1 / 100 * bought_at:  # 0.1 is stop loss
                data_index_buy_sell.append({"bought": i})
                break
            elif this_candle_earned < -0.05 / 100 * bought_at:  # 0.05 is stop loss
                break
            
    all_data_index_buy_sell.append(data_index_buy_sell)
    print(f"Length of the marked data: {len(data_index_buy_sell)}", f"total length: {len(candles)}")

Length of the marked data: 7335 total length: 17378
Length of the marked data: 8953 total length: 22663
Length of the marked data: 8672 total length: 21769
Length of the marked data: 8530 total length: 21830
Length of the marked data: 5902 total length: 15422


In [44]:
        # current_price = float(quotation_to_decimal(candle.close))
        # if current_price > float(quotation_to_decimal(candles[i - 1].close)):
        #     if len(data_index_buy_sell) == 0 or list(data_index_buy_sell[-1].keys())[0] == "sold":
        #         bought_at = float(quotation_to_decimal(candles[i - 1].close))
        #         data_index_buy_sell.append({"bought": i - 1})
        #     total_earn += current_price - float(quotation_to_decimal(candles[i - 1].close))
        # elif current_price < float(quotation_to_decimal(candles[i - 1].close)):
        #     data_index_buy_sell.append({"sold": i - 1})
        #     if total_earn > 0.1 / 100 * bought_at:  #0.1 / 100 * bought_at: bought_at * commission + current_price * commission
        #         total_sum_earned += total_earn
        #     else:
        #        data_index_buy_sell.pop(len(data_index_buy_sell) - 1)
        #     total_earn = 0

In [73]:
import pandas as pd

all_new_data_buy = list()
for data_index_buy_sell in all_data_index_buy_sell:
    new_data_buy = list()
    for i, data in enumerate(data_index_buy_sell):
        if list(data_index_buy_sell[i].keys())[0] != "sold":
            new_data_buy.append(data)
    all_new_data_buy.append(new_data_buy)

all_df_data = list()

for c_i, candles in enumerate(all_candles):
    df_data = {"index": [], "result": [], "close": [],
               "open": [], "high": [], "low": [],
               "volume": []}
    buy_counter = 0
    for i, candle in enumerate(list(candles)):
        df_data["index"].append(i)
        df_data["close"].append(float(quotation_to_decimal(candle.close)))
        df_data["open"].append(float(quotation_to_decimal(candle.open)))
        df_data["high"].append(float(quotation_to_decimal(candle.high)))
        df_data["low"].append(float(quotation_to_decimal(candle.low)))
        df_data["volume"].append(candle.volume)
        
        if i == all_new_data_buy[c_i][buy_counter]["bought"]:
            df_data["result"].append(1)
            buy_counter += 1 if buy_counter < len(all_new_data_buy[c_i]) - 1 else 0
        else:
            df_data["result"].append(0)
    all_df_data.append(df_data)
        
print(f"All different period data len: {len(all_df_data)}", f"true: {all_df_data[2]['result'].count(1)}", f"false: {all_df_data[2]['result'].count(0)}")

All different period data len: 5 true: 8672 false: 13097


# Add Strategy Signals To DF

In [74]:
from Strategies.StrategyAroon import StrategyAroon
from Strategies.StrategyStochRSI import StrategyStochRSI
from Strategies.StrategyEMA import StrategyEMA

# candle interval init (for this case is always 1 min)
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN

# init all strategies
EMA_down = StrategyEMA()
EMA_up = StrategyEMA()
EMA_flat = StrategyEMA()
EMA_short_up = StrategyEMA()
EMA_short_down = StrategyEMA()

RSI_down = StrategyRSI()
RSI_up = StrategyRSI()
RSI_flat = StrategyRSI()
RSI_short_up = StrategyRSI()
RSI_short_down = StrategyRSI()

MACD_down = StrategyMACD()
MACD_up = StrategyMACD()
MACD_flat = StrategyMACD()
MACD_short_up = StrategyMACD()
MACD_short_down = StrategyMACD()

BB_down = StrategyBB()
BB_up = StrategyBB()
BB_flat = StrategyBB()
BB_short_up = StrategyBB()
BB_short_down = StrategyBB()

ST_down = StrategyST()
ST_up = StrategyST()
ST_flat = StrategyST()
ST_short_up = StrategyST()
ST_short_down = StrategyST()

StochRSI_down = StrategyStochRSI()
StochRSI_up = StrategyStochRSI()
StochRSI_flat = StrategyStochRSI()
StochRSI_short_up = StrategyStochRSI()
StochRSI_short_down = StrategyStochRSI()

Aroon_down = StrategyAroon()
Aroon_up = StrategyAroon()
Aroon_flat = StrategyAroon()
Aroon_short_up = StrategyAroon()
Aroon_short_down = StrategyAroon()

OBV_down = StrategyOBV()
OBV_up = StrategyOBV()
OBV_flat = StrategyOBV()
OBV_short_up = StrategyOBV()
OBV_short_down = StrategyOBV()

strategies_blocks = [[EMA_down, EMA_up, EMA_flat, EMA_short_up, EMA_short_down], 
                    [RSI_down, RSI_up, RSI_flat, RSI_short_up, RSI_short_down], 
                    [MACD_down, MACD_up, MACD_flat, MACD_short_up, MACD_short_down], 
                    [BB_down, BB_up, BB_flat, BB_short_up, BB_short_down],
                    [StochRSI_down, StochRSI_up, StochRSI_flat, StochRSI_short_up, StochRSI_short_down],
                    [ST_down, ST_up, ST_flat, ST_short_up, ST_short_down],
                    [Aroon_down, Aroon_up, Aroon_flat, Aroon_short_up, Aroon_short_down],
                    [OBV_down, OBV_up, OBV_flat, OBV_short_up, OBV_short_down]]

for strategy_block in strategies_blocks:
    strategy_block[0].initialize_moving_avg_container(candles_down[:strategy_block[0].history_candles_length])
    strategy_block[1].initialize_moving_avg_container(candles_up[:strategy_block[1].history_candles_length])
    strategy_block[2].initialize_moving_avg_container(candles_flat[:strategy_block[2].history_candles_length])
    strategy_block[3].initialize_moving_avg_container(candles_flat[:strategy_block[3].history_candles_length])
    strategy_block[4].initialize_moving_avg_container(candles_flat[:strategy_block[4].history_candles_length])

print(f"Strategy blocks len: {len(strategies_blocks)}")     

Strategy blocks len: 8


In [75]:
from Strategies import StrategyABS

all_df_data_st = [{"past_EMA": [],
                "current_EMA": [],
                "current_RSI": [],
                "past_RSI": [],
                "past_MACD": [],
                "current_MACD": [],
                "current_OBV": [],
                "past_OBV": [],
                "upper_BB": [],
                "lower_BB": [],
                "%K_StochRSI": [],
                "upper_ST": [],
                "lower_ST": [],
                "ATR": [],
                "up_Aroon": [],
                "low_Aroon": []} for i in range(len(strategies_blocks[0]))]

def add_strategy_params(strategy_name: str, candle_index: int, history_candles_length: int, strategy: StrategyABS, current_layer: int) -> None:
            global all_df_data_st
            past = "past_" + strategy_name
            current = "current_" + strategy_name
            
            if candle_index == history_candles_length:
                [all_df_data_st[current_layer][past].append(None) for i in range(strategy.history_candles_length)]
                [all_df_data_st[current_layer][current].append(None) for i in range(strategy.history_candles_length)]
            params = strategy.get_candle_param(all_candles[str_i][candle_index])
            all_df_data_st[current_layer][past].append(params[0])
            all_df_data_st[current_layer][current].append(params[1])

for strategy_block in strategies_blocks:
    for str_i, strategy_type in enumerate(strategy_block):
        for candle_index in range(strategy_type.history_candles_length, len(all_candles[str_i])):
            if strategy_type.__class__.__name__ == "StrategyEMA":
                add_strategy_params("EMA", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyRSI":
                add_strategy_params("RSI", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyMACD":
                add_strategy_params("MACD", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyOBV":
                add_strategy_params("OBV", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyBB":
                if candle_index == strategy_type.history_candles_length:
                    [all_df_data_st[str_i]["upper_BB"].append(None) for i in range(strategy_type.history_candles_length)]
                    [all_df_data_st[str_i]["lower_BB"].append(None) for i in range(strategy_type.history_candles_length)]
                c_params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
                all_df_data_st[str_i]["upper_BB"].append(c_params[1])
                all_df_data_st[str_i]["lower_BB"].append(c_params[2])
            elif strategy_type.__class__.__name__ == "StrategyStochRSI":
                if candle_index == strategy_type.history_candles_length:
                    [all_df_data_st[str_i]["%K_StochRSI"].append(None) for i in range(strategy_type.history_candles_length)]
                c_params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
                all_df_data_st[str_i]["%K_StochRSI"].append(c_params[0])
            elif strategy_type.__class__.__name__ == "StrategyST":
                if candle_index == strategy_type.history_candles_length:
                    [all_df_data_st[str_i]["upper_ST"].append(None) for i in range(strategy_type.history_candles_length)]
                    [all_df_data_st[str_i]["lower_ST"].append(None) for i in range(strategy_type.history_candles_length)]
                    [all_df_data_st[str_i]["ATR"].append(None) for i in range(strategy_type.history_candles_length)]
                c_params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
                all_df_data_st[str_i]["upper_ST"].append(c_params[0])
                all_df_data_st[str_i]["lower_ST"].append(c_params[1])
                all_df_data_st[str_i]["ATR"].append(c_params[2])
            elif strategy_type.__class__.__name__ == "StrategyAroon":
                if candle_index == strategy_type.history_candles_length:
                    [all_df_data_st[str_i]["up_Aroon"].append(None) for i in range(strategy_type.history_candles_length)]
                    [all_df_data_st[str_i]["low_Aroon"].append(None) for i in range(strategy_type.history_candles_length)]
                c_params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
                all_df_data_st[str_i]["up_Aroon"].append(c_params[0])
                all_df_data_st[str_i]["low_Aroon"].append(c_params[1])

print(f"Small length check should be the length of candles: {len(all_df_data_st[2]['lower_BB'])}, candle len: {len(all_candles[2])}")

Small length check should be the length of candles: 21769, candle len: 21769


In [76]:
# convert to df
for i in range(len(all_df_data)):
    all_df_data[i] = pd.DataFrame(all_df_data[i])

for i in range(len(all_df_data)):
    for name in all_df_data_st[i]:
        all_df_data[i][name] = all_df_data_st[i][name]

biggest_gap = 0
for strategy_block in strategies_blocks:
    for strategy_type in strategy_block:
        if strategy_type.history_candles_length > biggest_gap:
            biggest_gap = strategy_type.history_candles_length

# delete first NaN elements
for i, df in enumerate(all_df_data):
    all_df_data[i] = df.drop(df.index[:biggest_gap])

concatenated_df = pd.DataFrame()
for i, data in enumerate(all_df_data):
    data.to_csv(f"./marked_data/result_with_parameters_{i}.csv", index=False)
    
all_df_data[0]

,index,result,close,open,high,low,volume,past_EMA,current_EMA,current_RSI,...,current_OBV,past_OBV,upper_BB,lower_BB,%K_StochRSI,upper_ST,lower_ST,ATR,up_Aroon,low_Aroon
200,200,1,265.12,265.00,265.34,264.62,65474,3.245200,3.409157,47.195712,...,-20408.066667,65474.0,271.441659,263.052341,2.558635,272.525571,268.662816,1.905844,28.0,92.0
201,201,1,264.22,265.11,265.23,264.14,83902,3.409157,3.629551,46.418695,...,-22386.066667,-83902.0,271.110899,262.786101,0.000000,272.525571,268.662816,1.918121,24.0,100.0
202,202,0,262.86,264.22,264.48,262.53,200461,3.629551,3.939523,45.280853,...,-40083.133333,-200461.0,270.537024,262.514976,0.000000,272.415738,268.662816,1.986913,20.0,100.0
203,203,0,263.69,262.86,263.85,262.82,77133,3.939523,4.142661,46.095365,...,-31488.066667,77133.0,269.931718,262.442282,17.887931,271.724355,268.662816,1.989785,16.0,96.0
204,204,0,264.50,263.69,264.74,263.58,71947,4.142661,4.251615,46.874850,...,-14430.200000,71947.0,269.336210,262.511790,35.344828,270.683398,268.662816,2.001133,12.0,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17373,17373,0,130.70,130.75,130.87,130.50,13611,-0.975887,-1.009690,53.147416,...,643.600000,0.0,130.764206,129.736794,41.176471,109.181222,276.175918,1.557147,16.0,28.0
17374,17374,0,130.80,130.70,130.98,130.61,17154,-1.009690,-1.047542,53.347018,...,1410.466667,17154.0,130.849445,129.715555,44.444444,109.181222,276.175918,1.544670,12.0,24.0
17375,17375,0,130.95,130.85,131.00,130.80,11660,-1.047542,-1.093248,53.646210,...,2514.133333,11660.0,130.871962,129.837038,48.717949,109.181222,276.175918,1.521692,100.0,20.0
17376,17376,1,131.30,130.85,131.50,130.74,37603,-1.093248,-1.163004,54.336387,...,5900.133333,37603.0,131.071448,129.737552,56.521739,109.181222,276.175918,1.537579,100.0,16.0
